In [8]:
%matplotlib inline
import os
import csv
import numpy as np
import re
import datetime
import fnmatch
from pathlib import Path

In [2]:
NotInclude = ['scratchPadText', '']

In [3]:
def phrasesTyped2(your_list):
    # Find entries in scratchpad that are the start and end of the phrase
    
    IndexNotInclude = list()
    
    # After every complete phrase, the next phrase is empyty '', and the first phrase is 'scratchPadText'
    for row_ind in range(0, len(your_list)):
        if your_list[row_ind][1] in NotInclude:
            if row_ind == 0:
                IndexNotInclude.append(row_ind) # first element is to be considered for the first phrase start time
                continue
            if your_list[row_ind][1] == your_list[row_ind-1][1]:
                continue
            IndexNotInclude.append(row_ind)
    
    for row_ind in range(0, len(your_list)):
        list_inner = your_list[row_ind]
        if list_inner[1] in NotInclude:
            IndexNotInclude.append(row_ind)
    
    # Remove consecutive entries with blank scratchpads

    IndexNotIncludeArray = np.asarray(IndexNotInclude)
    IndDiff = np.diff(IndexNotIncludeArray,1)
    
    ind = len(IndexNotInclude)-1
    while (ind>-1):
        if IndDiff[ind] == 1:
            for i_2 in range (ind,len(IndexNotInclude)):
                IndexNotInclude[i_2] = IndexNotInclude[i_2]-1
        
            del(IndexNotInclude[ind])
            del your_list[IndexNotIncludeArray[ind]]
        ind = ind - 1
        
        # Find the time taken for each phrase
    PhraseList = list()

    for indPhrase in range(0, len(IndexNotInclude)-1):
        startPhrase = IndexNotInclude[indPhrase]
        endPhrase = IndexNotInclude[indPhrase+1]
        headStartPhrase,sep,tail = your_list[startPhrase+1][0].partition('+')
        headEndPhrase,sep,tail = your_list[endPhrase-1][0].partition('+')
        timeTaken = datetime.datetime.strptime(re.sub('[:.T]','-',headEndPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f") - datetime.datetime.strptime(re.sub('[:.T]', '-', headStartPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
        phraseTyped = your_list[endPhrase-1][1]
        PhraseList.append([timeTaken, phraseTyped])
        
    #print(timeTaken)
    #print(PhraseList)
    
    # Compute the total time for the phrase and the total number of words typed during that time (1 word = 5 characters, including space)
    timeAdd = datetime.datetime.strptime('1000-1-1-0-00-00-000000',"%Y-%m-%d-%H-%M-%S-%f")
    characterLen = 0

    for indPhrase in range(0, len(IndexNotInclude)-1):
        timeAdd += PhraseList[indPhrase][0]
        characterLen += len(PhraseList[indPhrase][1])
    
    TimeTaken = (timeAdd.microsecond/1000000 + timeAdd.second)/60 + timeAdd.minute
    WPM = (characterLen)/(5*TimeTaken)
    #print("Typing speed is", WPM)

In [4]:
def timeTypingStart(userKeys):
    # From the user keys, find when the user actually starts typing, after having looked at the phrase and all the other 
    # function keys
    
    timeTypingStartInd = 0
    
    timeTypingStartIndList = list()
            
    timeUserTimeInd = 0
    
    ind = 0
    # Get start time of first trial
    
    while ind < len(userKeys):
        #print(len(userKeys[ind][1]))
        if len(userKeys[ind][1]) > 1:
            ind = ind + 1
        else:
            timeTypingStartInd = ind
            timeTypingStartIndList.append(ind)
            break
    
    #print(timeTypingStartInd)
    # Get every next phrase start timings
    while ind < len(userKeys):
        
        if userKeys[ind][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
            
            #timeTypingStartIndList.append(ind+1)
            for ind2 in range(ind+1, len(userKeys)):
                if len(userKeys[ind2][1]) > 1:
                    ind = ind + 1
                    continue
                elif userKeys[ind2][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
                    ind = ind + 1
                    continue
                else:
                    ind = ind2
                    timeTypingStartIndList.append(ind)
                    break
                    
        else:
            ind = ind + 1
            
    #print(timeTypingStartIndList)
    
    return timeTypingStartIndList

In [5]:
def phrasesTyped(your_list, timeList, userKeys, stimPhraseList):
    # Find entries in scratchpad (end of each trial)
    
    IndexNotInclude = list()
    PhraseList = list()
    TypingSpeed = list()

    # After every complete phrase, the next phrase is empyty ''
    for row_ind in range(0, len(your_list)):
        if your_list[row_ind][1] == '':
            if row_ind == 1:# first empty phrase needs to be not considered , at least for most of the subjects
                continue
            if your_list[row_ind][1] == your_list[row_ind-1][1]:
                continue
            IndexNotInclude.append(row_ind)
        
    if 'round robin scheduling' in stimPhraseList: # subj 'snk' saw the phrase and directly went to type the next phrase
        phraseRemoveInd = stimPhraseList.index('round robin scheduling')
        timeList.remove(timeList[phraseRemoveInd])
    
    # Find the time taken for each phrase  

    #print(len(timeList), len(stimPhraseList), len(IndexNotInclude))
    #print(stimPhraseList)
    if len(IndexNotInclude) != len(timeList) :
        #print(IndexNotInclude, timeList)
        print('error and both are not same')
        #print(IndexNotInclude)
        #print(scratchPad)
    else:
        
        for indPhrase in range(0, len(IndexNotInclude)):
            startPhrase = timeList[indPhrase]      
            endPhrase = IndexNotInclude[indPhrase]-1
            headStartPhrase,sep,tail = userKeys[startPhrase][0].partition('+')
            headEndPhrase,sep,tail = your_list[endPhrase][0].partition('+')
            timeTaken = datetime.datetime.strptime(re.sub('[:.T]','-',headEndPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f") - datetime.datetime.strptime(re.sub('[:.T]', '-', headStartPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            phraseTyped = your_list[endPhrase][1]
            PhraseList.append([timeTaken, phraseTyped])
#            print(startPhrase, endPhrase, headStartPhrase, headEndPhrase, timeTaken, phraseTyped)
            
        
    #print(timeTaken)
#    print(PhraseList)
    
    # Compute the total time for the phrase and the total number of words typed during that time (1 word = 5 characters, including space)
    timeAdd = datetime.datetime.strptime('1000-1-1-0-00-00-000000',"%Y-%m-%d-%H-%M-%S-%f")
    characterLen = 0
    
    for indPhrase in range(0, len(IndexNotInclude)):
        timeAdd += PhraseList[indPhrase][0]
        characterLen += len(PhraseList[indPhrase][1])
#        print(timeAdd, ': ', characterLen)
#    print(timeAdd, ': ', characterLen)
        
    TimeTaken = (timeAdd.microsecond/1000000 + timeAdd.second)/60 + timeAdd.minute
#    print(TimeTaken)
    WPM = (characterLen)/(5*TimeTaken)
    print("Typing speed is", WPM)
    
    return WPM

In [33]:
def AddDataToFile(metricComputed, Names, DataEvaluated):

    # Save the data
    fileIn = r'C:\DTU\Data\201805_HealthnRehab\data_summary.csv'

    if Path(fileIn).is_file():
        print(fileIn)
        fSize = os.path.getsize(fileIn) # or pd.read_excel(filename) for xls file

        # file exists
        # check if the given column exists
        with open(fileIn, 'r', newline='') as csvfileRead:
            if fSize > 0:
                fileOut = fileIn[:-4] + '1.csv'
                with open(fileOut, 'w', newline='') as csvfileWrite:
                    for row in csv.reader(csvfileRead):
                        if metricComputed in row:
                            # file exists and the metric is present in the file
                            print('This metric is already calculated and saved: ', metricComputed)
                            break
                        else:
                            # file exists but the metric is not present
                            print('adding the metric')
                            filewriter = csv.writer(csvfileWrite, delimiter = ',')
                            if len(row) > 1:
                                rowJoined = list()
                                for rowElement in row:
                                    rowJoined.append(rowElement)
                                
                                
                                print(rowJoined[0])
                                
                                if rowJoined[0] == 'subject_name':
                                    # write the titles of the metrics
                                    rowJoined.append(metricComputed)
                                    filewriter.writerow(rowJoined)
                                else:
                                    #filewriter.writerow([row, str(DataEvaluated[Names.index(row[0])])])
                                    rowJoined.append(str(DataEvaluated[Names.index(row[0])]))
                                    filewriter.writerow(rowJoined)
                                
                            else:
                                if row[0] == 'subject_name':
                                    # write the titles of the metrics
                                    filewriter.writerow([row, metricComputed])
                                else:
                                    filewriter.writerow([row, str(DataEvaluated[Names.index(row[0])])])
            
            
            else:
                fileOut = fileIn
                print('recreating a file')

                with open(fileSave, 'w', newline='') as csvfileWrite:
                    filewriter = csv.writer(csvfileWrite, delimiter=',')
                    filewriter.writerow(['subject_name', metricComputed])
                    for ind in enumerate(Names):
                        filewriter.writerow([Names[ind], DataEvaluated[ind[0]]])
                
    else:
        print('creating a file')
        a = re.compile('(?<=TypingData)(.*)(?=OptiKey)')
        with open(fileSave, 'w', newline='') as csvfileWrite:
            filewriter = csv.writer(csvfileWrite, delimiter=',')
            filewriter.writerow(['subject_name', metricComputed])
            for ind in enumerate(Names):
                subjName = a.findall(Names[ind[0]])
                filewriter.writerow([subjName[0], DataEvaluated[ind[0]]])
    

In [34]:
metricComputed = 'typing_speed'
dataFolderName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'
j = 0
TimeDwellAvg = 0
nSub = 0
TypingSpeed = list()
Names = list()

for root, dirs, subfolder in os.walk(dataFolderName):
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'notCompleted' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'joha' in root:
            continue
            
        scratchPad = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                     
        if scratchPad is None or userKeys is None or phraseLog is None:
            continue
        else:
            
            print(root)
            stimPhraseList = [key[1] for key in phraseLog]
            
            timeList = timeTypingStart(userKeys)
            
            if 'mcc' in root or 'be' in root: # The person looks at next phrase twice in the end, after 
                #'EXPERIMENT IS DONE' is shown
                timeList.remove(timeList[-1])
            speed = phrasesTyped(scratchPad, timeList, userKeys, stimPhraseList)
            
            TypingSpeed.append(speed)
            
            # save only the name of the subject and the date part of the file
            a = re.compile('(?<=TypingData)(.*)(?=OptiKey)')
            print(a.findall(root))
            Names.append(a.findall(root)[0])
            
if TypingSpeed:
    AddDataToFile(metricComputed, Names, TypingSpeed)
            

C:\DTU\Data\201805_HealthnRehab\TypingData\May15\akt_MS\OptiKeyLogs\2018-5-15-14-3-18
Typing speed is 5.780714684896461
['\\May15\\akt_MS\\']
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\be_DT\OptiKeyLogs\2018-5-15-14-51-13-notInclude
Typing speed is 1.8784331953931426
['\\May15\\be_DT\\']
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\jl_DT\OptiKeyLogs\2018-5-15-10-21-20
Typing speed is 4.090572633970403
['\\May15\\jl_DT\\']
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\KEA_MS\OptiKeyLogs\2018-5-15-12-31-49
Typing speed is 10.785735743678744
['\\May15\\KEA_MS\\']
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\lone_DT\OptiKeyLogs\2018-5-15-11-4-33
Typing speed is 4.103883820943128
['\\May15\\lone_DT\\']
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\mcc_MS\OptiKeyLogs\2018-5-15-12-7-17
Typing speed is 8.7788353031733
['\\May15\\mcc_MS\\']
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\MK_DT\OptiKeyLogs\2018-5-15-14-32-48
Typing speed is 4.625489993216377
['\\May15\\MK_DT